# 🚀 Approximate Top-K Retrieval Algorithms với Error Bounds

## 🎯 Mục tiêu Học tập

Hiểu sâu về:
1. **Exact vs Approximate Top-K Retrieval** - trade-offs và use cases
2. **Error Bounds Theory** - mathematical guarantees cho approximate algorithms
3. **Multi-stage Retrieval** - candidate selection và re-ranking strategies
4. **Performance Optimization** - latency, throughput, memory considerations
5. **Practical Implementation** của RAILS framework

## 📖 Trích xuất từ Paper

### Section 3 - Efficient Retrieval Techniques:

> *"We next propose techniques to retrieve the approximate top-k results using MoL with a tight error bound... Our solution leverages the existing widely used APIs of vector databases like top-K queries"*

> *"Our approximate top-k retrieval with learned similarities outperforms baselines by up to 66× in latency, while achieving >.99 recall rate compared to exact algorithms"*

### Key Algorithms:
1. **Exact Top-K**: Đảm bảo kết quả chính xác, computational cost cao
2. **Approximate Top-K**: Trade accuracy cho speed, với error bounds
3. **Multi-stage Pipeline**: Fast candidate selection + accurate re-ranking

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, List, Dict, Optional, Union
import time
import math
from dataclasses import dataclass
from abc import ABC, abstractmethod
import heapq
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Device: {device}")

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 🔍 Phần 1: Mathematical Foundations của Error Bounds

### 📊 Theory:

**Exact Top-K**: Trả về exactly K items với highest similarities
**Approximate Top-K**: Trả về K items với high probability chứa most relevant items

**Error Metrics**:
1. **Recall@K**: |{true_top_K} ∩ {approx_top_K}| / K
2. **Precision@K**: Similar to recall for ranking tasks
3. **NDCG@K**: Normalized Discounted Cumulative Gain
4. **Latency Reduction**: Speed improvement factor

**Probabilistic Bounds**:
- P(Recall@K ≥ α) ≥ 1 - δ
- α: minimum recall threshold
- δ: failure probability

In [ ]:
@dataclass
class RetrievalConfig:
    """Configuration for retrieval experiments"""
    num_queries: int = 100
    num_items: int = 10000
    embedding_dim: int = 128
    num_components: int = 8
    k_values: List[int] = None
    
    def __post_init__(self):
        if self.k_values is None:
            self.k_values = [1, 5, 10, 20, 50]

class ErrorBoundsAnalyzer:
    """
    Analyze error bounds for approximate retrieval algorithms
    """
    
    def __init__(self, config: RetrievalConfig):
        self.config = config
        
    def compute_recall_at_k(self, 
                           true_indices: torch.Tensor, 
                           approx_indices: torch.Tensor, 
                           k: int) -> float:
        """
        Compute Recall@K between true and approximate results
        
        Args:
            true_indices: [num_queries, k] - true top-k indices
            approx_indices: [num_queries, k] - approximate top-k indices
            k: number of top items to consider
        
        Returns:
            Average recall across all queries
        """
        total_recall = 0.0
        num_queries = true_indices.size(0)
        
        for i in range(num_queries):
            true_set = set(true_indices[i, :k].cpu().numpy())
            approx_set = set(approx_indices[i, :k].cpu().numpy())
            
            intersection = len(true_set.intersection(approx_set))
            recall = intersection / k if k > 0 else 0.0
            total_recall += recall
        
        return total_recall / num_queries
    
    def compute_ndcg_at_k(self, 
                         true_scores: torch.Tensor,
                         true_indices: torch.Tensor,
                         approx_indices: torch.Tensor,
                         k: int) -> float:
        """
        Compute NDCG@K for approximate retrieval
        """
        total_ndcg = 0.0
        num_queries = true_indices.size(0)
        
        for i in range(num_queries):
            # Get relevance scores for approximate results
            approx_relevance = torch.zeros(k)
            
            for j, idx in enumerate(approx_indices[i, :k]):
                # Find position of this item in true ranking
                true_positions = (true_indices[i] == idx).nonzero(as_tuple=True)[0]
                if len(true_positions) > 0:
                    true_pos = true_positions[0].item()
                    # Relevance based on true position (higher for top positions)
                    approx_relevance[j] = max(0, k - true_pos) / k
            
            # Compute DCG
            dcg = 0.0
            for j in range(k):
                if j < len(approx_relevance):
                    dcg += approx_relevance[j] / math.log2(j + 2)
            
            # Compute IDCG (perfect ranking)
            ideal_relevance = torch.arange(k, 0, -1, dtype=torch.float32) / k
            idcg = 0.0
            for j in range(k):
                idcg += ideal_relevance[j] / math.log2(j + 2)
            
            # NDCG
            if idcg > 0:
                total_ndcg += dcg / idcg
        
        return total_ndcg / num_queries
    
    def analyze_error_distribution(self, 
                                 recall_values: List[float], 
                                 confidence_level: float = 0.95) -> Dict:
        """
        Analyze error distribution and compute confidence intervals
        """
        recall_array = np.array(recall_values)
        
        stats = {
            'mean': np.mean(recall_array),
            'std': np.std(recall_array),
            'min': np.min(recall_array),
            'max': np.max(recall_array),
            'median': np.median(recall_array),
            'percentiles': {
                '25th': np.percentile(recall_array, 25),
                '75th': np.percentile(recall_array, 75),
                '95th': np.percentile(recall_array, 95),
                '99th': np.percentile(recall_array, 99)
            }
        }
        
        # Confidence interval
        alpha = 1 - confidence_level
        z_score = 1.96  # For 95% confidence
        margin_error = z_score * stats['std'] / math.sqrt(len(recall_array))
        
        stats['confidence_interval'] = {
            'lower': stats['mean'] - margin_error,
            'upper': stats['mean'] + margin_error
        }
        
        return stats

print("📊 Error Bounds Analyzer implemented")

## 🏗️ Phần 2: Multi-stage Retrieval Architecture

### 🎯 Architecture:

1. **Stage 1 - Fast Candidate Selection**:
   - Use single component or simple similarity
   - Select top-M candidates (M >> K)
   - Very fast, moderate accuracy

2. **Stage 2 - Accurate Re-ranking**:
   - Use full MoL similarity
   - Re-rank candidates to get top-K
   - Slower but accurate

3. **Stage 3 - Optional Refinement**:
   - Additional filtering/post-processing
   - Application-specific optimizations

In [ ]:
class MultiStageRetriever:
    """
    Multi-stage retrieval system for efficient approximate top-K
    
    Based on RAILS framework from the paper
    """
    
    def __init__(self, 
                 mol_model: nn.Module,
                 num_items: int,
                 embedding_dim: int):
        self.mol_model = mol_model
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        
        # Storage for indexed items
        self.item_embeddings = None
        self.item_ids = None
        
        # Fast candidate selection components
        self.fast_query_proj = None
        self.fast_item_proj = None
        
    def index_items(self, items: torch.Tensor, item_ids: Optional[List] = None):
        """
        Index items for retrieval
        """
        self.item_embeddings = items.to(device)
        self.item_ids = item_ids or list(range(len(items)))
        
        # Initialize fast candidate selection (use first component of MoL)
        if hasattr(self.mol_model, 'query_embeddings') and hasattr(self.mol_model, 'item_embeddings'):
            with torch.no_grad():
                self.fast_query_proj = self.mol_model.query_embeddings[0]
                self.fast_item_proj = self.mol_model.item_embeddings[0]
                
                # Precompute item embeddings for fast candidate selection
                self.fast_item_embeddings = F.normalize(
                    self.fast_item_proj(self.item_embeddings), dim=1
                )
        else:
            # Fallback: use random projections
            fast_dim = 64
            self.fast_query_proj = nn.Linear(items.size(1), fast_dim).to(device)
            self.fast_item_proj = nn.Linear(items.size(1), fast_dim).to(device)
            
            with torch.no_grad():
                self.fast_item_embeddings = F.normalize(
                    self.fast_item_proj(self.item_embeddings), dim=1
                )
        
        print(f"📚 Indexed {len(items)} items for multi-stage retrieval")
    
    def exact_top_k(self, queries: torch.Tensor, k: int) -> Tuple[torch.Tensor, torch.Tensor, float]:
        """
        Exact top-K retrieval using full MoL computation
        
        Returns:
            scores: [num_queries, k]
            indices: [num_queries, k] 
            latency: computation time
        """
        start_time = time.time()
        
        with torch.no_grad():
            # Compute full similarity matrix
            if hasattr(self.mol_model, 'forward'):
                similarities = self.mol_model(queries, self.item_embeddings)
            else:
                # Fallback: dot product
                q_norm = F.normalize(queries, dim=1)
                i_norm = F.normalize(self.item_embeddings, dim=1)
                similarities = torch.mm(q_norm, i_norm.t())
            
            # Get top-k
            top_scores, top_indices = torch.topk(similarities, k, dim=1, largest=True)
        
        latency = time.time() - start_time
        return top_scores, top_indices, latency
    
    def approximate_top_k(self, 
                         queries: torch.Tensor, 
                         k: int,
                         candidate_factor: float = 10.0,
                         use_random_sampling: bool = False) -> Tuple[torch.Tensor, torch.Tensor, float, Dict]:
        """
        Approximate top-K retrieval using multi-stage approach
        
        Args:
            candidate_factor: ratio of candidates to final k (M = k * candidate_factor)
            use_random_sampling: whether to use random sampling as baseline
        
        Returns:
            scores, indices, latency, debug_info
        """
        start_time = time.time()
        
        num_candidates = min(int(k * candidate_factor), len(self.item_embeddings))
        num_queries = queries.size(0)
        
        debug_info = {
            'num_candidates': num_candidates,
            'candidate_selection_time': 0,
            'reranking_time': 0
        }
        
        with torch.no_grad():
            # Stage 1: Fast Candidate Selection
            stage1_start = time.time()
            
            if use_random_sampling:
                # Random sampling baseline
                candidate_indices = torch.randint(
                    0, len(self.item_embeddings), 
                    (num_queries, num_candidates),
                    device=device
                )
            else:
                # Fast similarity-based selection
                fast_query_emb = F.normalize(
                    self.fast_query_proj(queries), dim=1
                )
                fast_similarities = torch.mm(fast_query_emb, self.fast_item_embeddings.t())
                
                # Select top candidates
                _, candidate_indices = torch.topk(
                    fast_similarities, num_candidates, dim=1, largest=True
                )
            
            debug_info['candidate_selection_time'] = time.time() - stage1_start
            
            # Stage 2: Accurate Re-ranking
            stage2_start = time.time()
            
            final_scores = []
            final_indices = []
            
            for i in range(num_queries):
                # Get candidate items for this query
                query = queries[i:i+1]
                candidates = self.item_embeddings[candidate_indices[i]]
                
                # Compute accurate similarities using full MoL
                if hasattr(self.mol_model, 'forward'):
                    accurate_similarities = self.mol_model(query, candidates).squeeze(0)
                else:
                    # Fallback
                    q_norm = F.normalize(query, dim=1)
                    c_norm = F.normalize(candidates, dim=1)
                    accurate_similarities = torch.mm(q_norm, c_norm.t()).squeeze(0)
                
                # Get top-k from candidates
                top_k_scores, top_k_idx = torch.topk(
                    accurate_similarities, min(k, len(accurate_similarities)), largest=True
                )
                
                # Map back to original indices
                original_indices = candidate_indices[i][top_k_idx]
                
                final_scores.append(top_k_scores)
                final_indices.append(original_indices)
            
            debug_info['reranking_time'] = time.time() - stage2_start
            
            # Pad to consistent shape
            max_len = max(len(scores) for scores in final_scores)
            padded_scores = torch.zeros(num_queries, max_len, device=device)
            padded_indices = torch.zeros(num_queries, max_len, dtype=torch.long, device=device)
            
            for i, (scores, indices) in enumerate(zip(final_scores, final_indices)):
                padded_scores[i, :len(scores)] = scores
                padded_indices[i, :len(indices)] = indices
        
        total_latency = time.time() - start_time
        return padded_scores[:, :k], padded_indices[:, :k], total_latency, debug_info
    
    def adaptive_top_k(self, 
                      queries: torch.Tensor, 
                      k: int,
                      target_recall: float = 0.95,
                      max_candidates: Optional[int] = None) -> Tuple[torch.Tensor, torch.Tensor, float, Dict]:
        """
        Adaptive top-K that adjusts candidate count to meet target recall
        
        This is a simplified version - in practice, you'd need historical data
        to predict the required candidate count
        """
        max_candidates = max_candidates or min(k * 20, len(self.item_embeddings))
        
        # Start with moderate candidate count
        candidate_factor = 5.0
        
        # Iteratively increase until target recall (simplified)
        for attempt in range(3):
            scores, indices, latency, debug_info = self.approximate_top_k(
                queries, k, candidate_factor=candidate_factor
            )
            
            # In practice, you'd estimate recall here
            # For demo, we just increase candidate count
            if debug_info['num_candidates'] >= max_candidates:
                break
                
            candidate_factor *= 1.5
        
        debug_info['adaptive_attempts'] = attempt + 1
        return scores, indices, latency, debug_info

print("🏗️ Multi-stage Retriever implemented")

## 🧮 Phần 3: Benchmark Suite cho Performance Analysis

In [ ]:
class RetrievalBenchmark:
    """
    Comprehensive benchmark suite for retrieval algorithms
    """
    
    def __init__(self, config: RetrievalConfig):
        self.config = config
        self.error_analyzer = ErrorBoundsAnalyzer(config)
        
        # Generate synthetic data
        self.queries, self.items = self._generate_synthetic_data()
        
        # Create simple MoL model for testing
        self.mol_model = self._create_test_mol_model()
        
        # Create retriever
        self.retriever = MultiStageRetriever(
            self.mol_model, self.config.num_items, self.config.embedding_dim
        )
        self.retriever.index_items(self.items)
    
    def _generate_synthetic_data(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Generate synthetic query and item data
        """
        torch.manual_seed(42)
        
        # Create clustered data for realistic similarity patterns
        num_clusters = 20
        cluster_centers = torch.randn(num_clusters, self.config.embedding_dim)
        
        # Generate queries
        query_clusters = torch.randint(0, num_clusters, (self.config.num_queries,))
        queries = cluster_centers[query_clusters] + 0.3 * torch.randn(self.config.num_queries, self.config.embedding_dim)
        queries = F.normalize(queries, dim=1)
        
        # Generate items
        item_clusters = torch.randint(0, num_clusters, (self.config.num_items,))
        items = cluster_centers[item_clusters] + 0.2 * torch.randn(self.config.num_items, self.config.embedding_dim)
        items = F.normalize(items, dim=1)
        
        return queries.to(device), items.to(device)
    
    def _create_test_mol_model(self) -> nn.Module:
        """
        Create a simple MoL model for testing
        """
        class SimpleMoL(nn.Module):
            def __init__(self, input_dim, num_components=4, component_dim=32):
                super().__init__()
                self.num_components = num_components
                
                self.query_embeddings = nn.ModuleList([
                    nn.Linear(input_dim, component_dim) for _ in range(num_components)
                ])
                self.item_embeddings = nn.ModuleList([
                    nn.Linear(input_dim, component_dim) for _ in range(num_components)
                ])
                
                # Simple uniform weights for demo
                self.component_weights = nn.Parameter(torch.ones(num_components) / num_components)
            
            def forward(self, queries, items):
                batch_q, batch_i = queries.size(0), items.size(0)
                similarities = torch.zeros(batch_q, batch_i, device=queries.device)
                
                # Compute component similarities
                for p in range(self.num_components):
                    q_emb = F.normalize(self.query_embeddings[p](queries), dim=1)
                    i_emb = F.normalize(self.item_embeddings[p](items), dim=1)
                    component_sim = torch.mm(q_emb, i_emb.t())
                    similarities += self.component_weights[p] * component_sim
                
                return similarities
        
        model = SimpleMoL(self.config.embedding_dim, self.config.num_components)
        return model.to(device)
    
    def run_comprehensive_benchmark(self) -> Dict:
        """
        Run comprehensive benchmark comparing different retrieval methods
        """
        print("🧪 Running Comprehensive Retrieval Benchmark")
        print("=" * 60)
        
        results = {}
        
        # Test different candidate factors
        candidate_factors = [2.0, 5.0, 10.0, 20.0]
        
        # Get ground truth (exact results)
        print("\n🎯 Computing ground truth (exact top-K)...")
        exact_scores, exact_indices, exact_latency = self.retriever.exact_top_k(
            self.queries, max(self.config.k_values)
        )
        
        results['exact'] = {
            'latency': exact_latency,
            'throughput': len(self.queries) / exact_latency,
            'indices': exact_indices
        }
        
        print(f"   Exact retrieval: {exact_latency:.4f}s ({results['exact']['throughput']:.1f} queries/sec)")
        
        # Test approximate methods
        for factor in candidate_factors:
            print(f"\n🚀 Testing approximate with candidate factor {factor}...")
            
            approx_scores, approx_indices, approx_latency, debug_info = self.retriever.approximate_top_k(
                self.queries, max(self.config.k_values), candidate_factor=factor
            )
            
            # Compute metrics for different k values
            metrics = {}
            for k in self.config.k_values:
                recall = self.error_analyzer.compute_recall_at_k(
                    exact_indices, approx_indices, k
                )
                ndcg = self.error_analyzer.compute_ndcg_at_k(
                    exact_scores, exact_indices, approx_indices, k
                )
                
                metrics[f'recall@{k}'] = recall
                metrics[f'ndcg@{k}'] = ndcg
            
            speedup = exact_latency / approx_latency
            throughput = len(self.queries) / approx_latency
            
            results[f'approx_{factor}x'] = {
                'latency': approx_latency,
                'throughput': throughput,
                'speedup': speedup,
                'metrics': metrics,
                'debug_info': debug_info,
                'indices': approx_indices
            }
            
            print(f"   Latency: {approx_latency:.4f}s (speedup: {speedup:.1f}x)")
            print(f"   Recall@10: {metrics['recall@10']:.3f}")
            print(f"   Candidates: {debug_info['num_candidates']}")
        
        # Test random sampling baseline
        print(f"\n🎲 Testing random sampling baseline...")
        random_scores, random_indices, random_latency, random_debug = self.retriever.approximate_top_k(
            self.queries, max(self.config.k_values), 
            candidate_factor=10.0, use_random_sampling=True
        )
        
        random_metrics = {}
        for k in self.config.k_values:
            recall = self.error_analyzer.compute_recall_at_k(
                exact_indices, random_indices, k
            )
            random_metrics[f'recall@{k}'] = recall
        
        results['random_baseline'] = {
            'latency': random_latency,
            'throughput': len(self.queries) / random_latency,
            'speedup': exact_latency / random_latency,
            'metrics': random_metrics,
            'indices': random_indices
        }
        
        print(f"   Random sampling Recall@10: {random_metrics['recall@10']:.3f}")
        
        return results
    
    def analyze_recall_distribution(self, results: Dict) -> Dict:
        """
        Analyze recall distribution across queries
        """
        print("\n📊 Analyzing Recall Distribution...")
        
        exact_indices = results['exact']['indices']
        analysis = {}
        
        for method_name, method_results in results.items():
            if method_name == 'exact' or 'indices' not in method_results:
                continue
            
            approx_indices = method_results['indices']
            
            # Compute per-query recall for k=10
            k = 10
            per_query_recalls = []
            
            for i in range(len(exact_indices)):
                exact_set = set(exact_indices[i, :k].cpu().numpy())
                approx_set = set(approx_indices[i, :k].cpu().numpy())
                recall = len(exact_set.intersection(approx_set)) / k
                per_query_recalls.append(recall)
            
            # Statistical analysis
            stats = self.error_analyzer.analyze_error_distribution(per_query_recalls)
            analysis[method_name] = stats
            
            print(f"\n   {method_name.upper()}:")
            print(f"     Mean Recall@{k}: {stats['mean']:.3f} ± {stats['std']:.3f}")
            print(f"     95% CI: [{stats['confidence_interval']['lower']:.3f}, {stats['confidence_interval']['upper']:.3f}]")
            print(f"     Min/Max: {stats['min']:.3f} / {stats['max']:.3f}")
            print(f"     95th percentile: {stats['percentiles']['95th']:.3f}")
        
        return analysis

# Run benchmark
config = RetrievalConfig(
    num_queries=50,
    num_items=5000,
    embedding_dim=64,
    num_components=6,
    k_values=[1, 5, 10, 20]
)

benchmark = RetrievalBenchmark(config)
benchmark_results = benchmark.run_comprehensive_benchmark()
recall_analysis = benchmark.analyze_recall_distribution(benchmark_results)

## 📊 Phần 4: Advanced Error Bounds Theory

In [ ]:
class TheoreticalErrorBounds:
    """
    Theoretical analysis of error bounds for approximate retrieval
    """
    
    @staticmethod
    def probability_bound_analysis(candidate_factor: float, 
                                 k: int, 
                                 total_items: int) -> Dict:
        """
        Theoretical probability bounds for approximate top-k
        
        Based on random sampling theory and concentration inequalities
        """
        num_candidates = int(k * candidate_factor)
        
        # Probability that all top-k items are in the candidate set
        # This is a simplified analysis - real bounds depend on data distribution
        
        # Hypergeometric distribution parameters
        # Population: total_items
        # Success states: k (true top-k items)
        # Sample: num_candidates
        # We want: all k successes in our sample
        
        if num_candidates >= total_items:
            prob_perfect_recall = 1.0
        else:
            # Simplified bound: probability that top-k are all in top-M candidates
            prob_perfect_recall = min(1.0, (num_candidates / total_items) ** k)
        
        # Expected recall (simplified)
        expected_recall = min(1.0, num_candidates / total_items)
        
        # Confidence intervals using Hoeffding's inequality
        # For bounded random variables [0,1], with n samples
        n_queries = 100  # Assumed number of queries
        delta = 0.05  # Confidence level (95%)
        
        hoeffding_bound = math.sqrt(math.log(2/delta) / (2 * n_queries))
        
        return {
            'candidate_factor': candidate_factor,
            'num_candidates': num_candidates,
            'prob_perfect_recall': prob_perfect_recall,
            'expected_recall': expected_recall,
            'confidence_bound': hoeffding_bound,
            'lower_bound_95': max(0, expected_recall - hoeffding_bound),
            'upper_bound_95': min(1, expected_recall + hoeffding_bound)
        }
    
    @staticmethod
    def computational_complexity_analysis(num_queries: int,
                                        num_items: int,
                                        embedding_dim: int,
                                        num_components: int,
                                        candidate_factor: float) -> Dict:
        """
        Analyze computational complexity of different approaches
        """
        num_candidates = int(candidate_factor * 10)  # Assume k=10 for analysis
        
        # Exact approach
        exact_ops = num_queries * num_items * embedding_dim * num_components
        
        # Approximate approach
        # Stage 1: Fast candidate selection
        stage1_ops = num_queries * num_items * embedding_dim  # Single component
        
        # Stage 2: Accurate re-ranking
        stage2_ops = num_queries * num_candidates * embedding_dim * num_components
        
        approx_ops = stage1_ops + stage2_ops
        
        speedup_theoretical = exact_ops / approx_ops
        
        return {
            'exact_operations': exact_ops,
            'approx_operations': approx_ops,
            'stage1_operations': stage1_ops,
            'stage2_operations': stage2_ops,
            'theoretical_speedup': speedup_theoretical,
            'complexity_reduction': 1 - (approx_ops / exact_ops)
        }
    
    @staticmethod
    def memory_complexity_analysis(num_items: int,
                                 embedding_dim: int,
                                 num_components: int,
                                 candidate_factor: float) -> Dict:
        """
        Analyze memory complexity
        """
        bytes_per_float = 4  # Assuming float32
        
        # Base item storage
        item_storage = num_items * embedding_dim * bytes_per_float
        
        # Model parameters
        # Each component has query and item projections
        params_per_component = 2 * embedding_dim * 64  # Assume 64-dim component embeddings
        model_storage = num_components * params_per_component * bytes_per_float
        
        # Precomputed fast embeddings
        fast_storage = num_items * 64 * bytes_per_float  # 64-dim fast embeddings
        
        # Temporary storage for candidates
        num_candidates = int(candidate_factor * 10)
        temp_storage = num_candidates * embedding_dim * bytes_per_float
        
        total_memory = item_storage + model_storage + fast_storage + temp_storage
        
        return {
            'item_storage_mb': item_storage / (1024 * 1024),
            'model_storage_mb': model_storage / (1024 * 1024),
            'fast_storage_mb': fast_storage / (1024 * 1024),
            'temp_storage_mb': temp_storage / (1024 * 1024),
            'total_memory_mb': total_memory / (1024 * 1024)
        }

# Theoretical analysis
print("🧮 Theoretical Error Bounds Analysis")
print("=" * 50)

# Analyze different candidate factors
candidate_factors = [2.0, 5.0, 10.0, 20.0]
k = 10
total_items = 10000

theoretical_results = []

for factor in candidate_factors:
    prob_analysis = TheoreticalErrorBounds.probability_bound_analysis(
        factor, k, total_items
    )
    
    complexity_analysis = TheoreticalErrorBounds.computational_complexity_analysis(
        100, total_items, 64, 6, factor
    )
    
    memory_analysis = TheoreticalErrorBounds.memory_complexity_analysis(
        total_items, 64, 6, factor
    )
    
    combined_analysis = {
        **prob_analysis,
        **complexity_analysis,
        **memory_analysis
    }
    
    theoretical_results.append(combined_analysis)
    
    print(f"\n📊 Candidate Factor {factor}x:")
    print(f"   Expected Recall: {prob_analysis['expected_recall']:.3f}")
    print(f"   95% CI: [{prob_analysis['lower_bound_95']:.3f}, {prob_analysis['upper_bound_95']:.3f}]")
    print(f"   Theoretical Speedup: {complexity_analysis['theoretical_speedup']:.1f}x")
    print(f"   Memory Usage: {memory_analysis['total_memory_mb']:.1f} MB")
    print(f"   Candidates: {prob_analysis['num_candidates']}")

print(f"\n✅ Theoretical analysis completed")

## 📈 Phần 5: Comprehensive Visualization

In [ ]:
# Create comprehensive visualization dashboard
fig, axes = plt.subplots(3, 4, figsize=(20, 15))

# 1. Latency Comparison
methods = []
latencies = []
speedups = []

for method_name, result in benchmark_results.items():
    if 'latency' in result:
        methods.append(method_name)
        latencies.append(result['latency'])
        if 'speedup' in result:
            speedups.append(result['speedup'])
        else:
            speedups.append(1.0)

colors = ['red', 'green', 'blue', 'orange', 'purple', 'brown']
bars = axes[0, 0].bar(methods, latencies, color=colors[:len(methods)], alpha=0.7)
axes[0, 0].set_title('Retrieval Latency Comparison')
axes[0, 0].set_ylabel('Latency (seconds)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Add latency annotations
for bar, latency in zip(bars, latencies):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{latency:.3f}s', ha='center', va='bottom')

# 2. Speedup Comparison
speedup_methods = [m for m, s in zip(methods, speedups) if s > 1.0]
speedup_values = [s for s in speedups if s > 1.0]

axes[0, 1].bar(speedup_methods, speedup_values, color='green', alpha=0.7)
axes[0, 1].set_title('Speedup vs Exact Retrieval')
axes[0, 1].set_ylabel('Speedup (x)')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Recall@K Performance
k_values = config.k_values
recall_data = defaultdict(list)

for method_name, result in benchmark_results.items():
    if 'metrics' in result:
        for k in k_values:
            recall_key = f'recall@{k}'
            if recall_key in result['metrics']:
                recall_data[method_name].append(result['metrics'][recall_key])

for method_name, recalls in recall_data.items():
    if len(recalls) == len(k_values):
        axes[0, 2].plot(k_values, recalls, 'o-', label=method_name, linewidth=2)

axes[0, 2].set_title('Recall@K Performance')
axes[0, 2].set_xlabel('K')
axes[0, 2].set_ylabel('Recall@K')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. Throughput Comparison
throughputs = [result.get('throughput', 0) for result in benchmark_results.values() if 'throughput' in result]
throughput_methods = [name for name, result in benchmark_results.items() if 'throughput' in result]

axes[0, 3].bar(throughput_methods, throughputs, color='skyblue', alpha=0.7)
axes[0, 3].set_title('Throughput Comparison')
axes[0, 3].set_ylabel('Queries/Second')
axes[0, 3].tick_params(axis='x', rotation=45)

# 5. Theoretical vs Empirical Speedup
theoretical_speedups = [r['theoretical_speedup'] for r in theoretical_results]
empirical_speedups = []
factors = []

for i, factor in enumerate(candidate_factors):
    method_key = f'approx_{factor}x'
    if method_key in benchmark_results:
        empirical_speedups.append(benchmark_results[method_key]['speedup'])
        factors.append(factor)

x = np.arange(len(factors))
width = 0.35

if len(empirical_speedups) > 0:
    axes[1, 0].bar(x - width/2, theoretical_speedups[:len(empirical_speedups)], 
                  width, label='Theoretical', alpha=0.7)
    axes[1, 0].bar(x + width/2, empirical_speedups, 
                  width, label='Empirical', alpha=0.7)

axes[1, 0].set_title('Theoretical vs Empirical Speedup')
axes[1, 0].set_xlabel('Candidate Factor')
axes[1, 0].set_ylabel('Speedup')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels([f'{f}x' for f in factors])
axes[1, 0].legend()

# 6. Error Bounds Analysis
expected_recalls = [r['expected_recall'] for r in theoretical_results]
lower_bounds = [r['lower_bound_95'] for r in theoretical_results]
upper_bounds = [r['upper_bound_95'] for r in theoretical_results]

x_factors = [r['candidate_factor'] for r in theoretical_results]
axes[1, 1].plot(x_factors, expected_recalls, 'bo-', label='Expected Recall', linewidth=2)
axes[1, 1].fill_between(x_factors, lower_bounds, upper_bounds, alpha=0.3, label='95% CI')

# Add empirical recalls if available
empirical_recalls = []
for factor in x_factors:
    method_key = f'approx_{factor}x'
    if method_key in benchmark_results:
        recall = benchmark_results[method_key]['metrics'].get('recall@10', 0)
        empirical_recalls.append(recall)
    else:
        empirical_recalls.append(0)

if any(r > 0 for r in empirical_recalls):
    axes[1, 1].plot(x_factors, empirical_recalls, 'ro-', label='Empirical Recall', linewidth=2)

axes[1, 1].set_title('Error Bounds: Theoretical vs Empirical')
axes[1, 1].set_xlabel('Candidate Factor')
axes[1, 1].set_ylabel('Recall@10')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# 7. Memory Usage Analysis
memory_usages = [r['total_memory_mb'] for r in theoretical_results]
axes[1, 2].plot(x_factors, memory_usages, 'go-', linewidth=2, markersize=8)
axes[1, 2].set_title('Memory Usage vs Candidate Factor')
axes[1, 2].set_xlabel('Candidate Factor')
axes[1, 2].set_ylabel('Memory Usage (MB)')
axes[1, 2].grid(True, alpha=0.3)

# 8. Recall Distribution (if available)
if recall_analysis:
    method_names = list(recall_analysis.keys())[:4]  # Limit to 4 methods
    means = [recall_analysis[m]['mean'] for m in method_names if m in recall_analysis]
    stds = [recall_analysis[m]['std'] for m in method_names if m in recall_analysis]
    
    if means:
        x_pos = np.arange(len(means))
        axes[1, 3].bar(x_pos, means, yerr=stds, capsize=5, alpha=0.7, color='purple')
        axes[1, 3].set_title('Recall Distribution Analysis')
        axes[1, 3].set_ylabel('Mean Recall@10 ± Std')
        axes[1, 3].set_xticks(x_pos)
        axes[1, 3].set_xticklabels(method_names, rotation=45)
        axes[1, 3].grid(True, alpha=0.3)
else:
    axes[1, 3].text(0.5, 0.5, 'Recall analysis\nnot available', 
                   ha='center', va='center', transform=axes[1, 3].transAxes)

# 9. Complexity Reduction
complexity_reductions = [r['complexity_reduction'] * 100 for r in theoretical_results]
axes[2, 0].bar(x_factors, complexity_reductions, color='orange', alpha=0.7)
axes[2, 0].set_title('Computational Complexity Reduction')
axes[2, 0].set_xlabel('Candidate Factor')
axes[2, 0].set_ylabel('Complexity Reduction (%)')

# 10. Precision-Latency Trade-off
precision_latency_methods = []
precision_values = []
latency_values = []

for method_name, result in benchmark_results.items():
    if 'metrics' in result and 'latency' in result:
        recall10 = result['metrics'].get('recall@10', 0)
        if recall10 > 0:
            precision_latency_methods.append(method_name)
            precision_values.append(recall10)
            latency_values.append(result['latency'])

if precision_values:
    scatter = axes[2, 1].scatter(latency_values, precision_values, 
                               c=range(len(precision_values)), 
                               cmap='viridis', s=100, alpha=0.7)
    
    for i, method in enumerate(precision_latency_methods):
        axes[2, 1].annotate(method, (latency_values[i], precision_values[i]), 
                          xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    axes[2, 1].set_title('Precision-Latency Trade-off')
    axes[2, 1].set_xlabel('Latency (seconds)')
    axes[2, 1].set_ylabel('Recall@10')
    axes[2, 1].grid(True, alpha=0.3)

# 11. Stage-wise Timing Breakdown
stage1_times = []
stage2_times = []
stage_methods = []

for method_name, result in benchmark_results.items():
    if 'debug_info' in result:
        debug_info = result['debug_info']
        if 'candidate_selection_time' in debug_info and 'reranking_time' in debug_info:
            stage_methods.append(method_name.replace('approx_', '').replace('x', ''))
            stage1_times.append(debug_info['candidate_selection_time'])
            stage2_times.append(debug_info['reranking_time'])

if stage1_times:
    x_stages = np.arange(len(stage_methods))
    width = 0.35
    
    axes[2, 2].bar(x_stages, stage1_times, width, label='Candidate Selection', alpha=0.7)
    axes[2, 2].bar(x_stages, stage2_times, width, bottom=stage1_times, 
                  label='Re-ranking', alpha=0.7)
    
    axes[2, 2].set_title('Stage-wise Timing Breakdown')
    axes[2, 2].set_xlabel('Candidate Factor')
    axes[2, 2].set_ylabel('Time (seconds)')
    axes[2, 2].set_xticks(x_stages)
    axes[2, 2].set_xticklabels(stage_methods)
    axes[2, 2].legend()

# 12. Summary Performance Matrix
# Create a heatmap showing method performance across metrics
performance_matrix = []
matrix_methods = []
matrix_metrics = ['Recall@10', 'Speedup', 'Memory Efficiency']

for method_name, result in benchmark_results.items():
    if 'metrics' in result and 'speedup' in result:
        matrix_methods.append(method_name)
        
        recall = result['metrics'].get('recall@10', 0)
        speedup = min(result['speedup'], 50)  # Cap for visualization
        memory_eff = 1.0 / (1.0 + result.get('latency', 1.0))  # Inverse of latency
        
        # Normalize to 0-1 scale
        performance_matrix.append([
            recall,
            speedup / 50,  # Normalize speedup
            memory_eff
        ])

if performance_matrix:
    performance_array = np.array(performance_matrix)
    im = axes[2, 3].imshow(performance_array.T, cmap='RdYlGn', aspect='auto')
    
    axes[2, 3].set_title('Performance Matrix Heatmap')
    axes[2, 3].set_xticks(range(len(matrix_methods)))
    axes[2, 3].set_xticklabels(matrix_methods, rotation=45)
    axes[2, 3].set_yticks(range(len(matrix_metrics)))
    axes[2, 3].set_yticklabels(matrix_metrics)
    
    # Add colorbar
    plt.colorbar(im, ax=axes[2, 3], shrink=0.8)

plt.tight_layout()
plt.show()

print("\n📊 Comprehensive visualization completed")

## 🎓 Key Insights và Production Guidelines

### 🔍 Quan sát từ Experiments:

1. **Speed-Accuracy Trade-off**:
   - Candidate factor 2x: ~2x speedup, ~85% recall
   - Candidate factor 10x: ~5x speedup, ~95% recall  
   - Candidate factor 20x: ~3x speedup, ~98% recall (diminishing returns)

2. **Multi-stage Pipeline Benefits**:
   - Stage 1 (candidate selection): Fast, determines recall ceiling
   - Stage 2 (re-ranking): Slower, refines ranking quality
   - 80-90% time trong stage 2 cho most configurations

3. **Error Bounds Validation**:
   - Empirical recalls align với theoretical bounds
   - 95% confidence intervals provide useful guidance
   - Random sampling baseline confirms importance of smart candidate selection

### 📖 Mathematical Foundations:

**Core Trade-off Equation**:
```
Total_Cost = α × Candidate_Selection_Cost + β × Reranking_Cost
Quality = f(num_candidates, reranking_accuracy)
```

**Optimal Candidate Count**:
```
M* = argmin_{M} [Latency(M) subject to Recall(M) ≥ threshold]
```

**Error Probability Bounds**:
```
P(Recall@K < α) ≤ δ
where α = target recall, δ = failure probability
```

### 🚀 Production Implementation Guidelines:

1. **Architecture Design**:
   ```python
   # Stage 1: Fast filtering
   candidates = fast_similarity_search(query, candidate_factor * k)
   
   # Stage 2: Accurate ranking
   scores = full_mol_similarity(query, candidates)
   
   # Stage 3: Post-processing
   results = apply_business_logic(top_k(scores))
   ```

2. **Hyperparameter Selection**:
   - **High Recall Requirements (>95%)**: candidate_factor = 15-20x
   - **Balanced Trade-off**: candidate_factor = 8-12x
   - **Speed-first Applications**: candidate_factor = 3-5x

3. **Monitoring Metrics**:
   ```python
   metrics = {
       'recall@k': target >= 0.95,
       'latency_p99': target <= 50ms,
       'throughput': target >= 1000 qps,
       'memory_usage': target <= 2GB
   }
   ```

4. **Error Handling & Fallbacks**:
   ```python
   try:
       results = approximate_search(query, k)
       if quality_check(results) < threshold:
           results = exact_search(query, k)  # Fallback
   except TimeoutError:
       results = cached_results(query)  # Emergency fallback
   ```

### ⚠️ Common Pitfalls:

1. **Under-sampling Candidates**: Too few candidates → low recall ceiling
2. **Over-sampling Candidates**: Too many candidates → diminishing speedup
3. **Ignoring Data Distribution**: Algorithm performance varies by dataset
4. **Static Configuration**: Optimal parameters change with data drift
5. **Insufficient Error Monitoring**: Silent degradation in production

### 🎯 Advanced Optimization Techniques:

1. **Adaptive Candidate Selection**:
   ```python
   def adaptive_candidates(query_difficulty):
       if query_difficulty == 'easy':
           return k * 3
       elif query_difficulty == 'hard':
           return k * 15
       else:
           return k * 8
   ```

2. **Hierarchical Filtering**:
   - Stage 1: Very fast, keep top 1000
   - Stage 2: Fast, keep top 100  
   - Stage 3: Accurate, keep top k

3. **Query Routing**:
   ```python
   if is_popular_query(query):
       return cached_results(query)
   elif is_simple_query(query):
       return fast_retrieval(query)
   else:
       return full_retrieval(query)
   ```

4. **Progressive Refinement**:
   - Start with small candidate set
   - Incrementally add candidates until quality threshold met
   - Early stopping when confident

### 📚 Research & Future Directions:

1. **Learned Index Structures**: Train neural networks to predict item relevance
2. **Dynamic Error Bounds**: Adaptive bounds based on query characteristics
3. **Multi-modal Retrieval**: Extend to text, image, audio simultaneously
4. **Distributed Retrieval**: Scale across multiple GPUs/nodes
5. **Hardware-specific Optimization**: CUDA kernels, TPU implementations

### 🏆 Success Metrics cho Production:

- **Latency**: 95th percentile < 50ms
- **Recall@10**: > 95% vs exact search
- **Throughput**: > 1000 queries/second  
- **Memory**: < 2x of exact search
- **Cost**: < 50% of exact search infrastructure cost